In [1]:
import torch
import torch.nn as nn 

In [15]:
## 首先搭建残差块 resnet block
## 残差块属于类，网络需要多次调用该block
## 该类属于神经网络模块调用，所以需继承nn.Module作为父类

class ResidualBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        
        # 第一个卷积块
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
        # 第二个卷积块
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # 残差边的通道对齐块
        self.shortcut = nn.Sequential()
        
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                                       nn.BatchNorm2d(out_channels))
        
        
    def forward(self, inputs):
        
        # 该残差块的输入
        identity = inputs
        
        # 输入经过第一个卷积层
        outputs = self.conv1(inputs)
        outputs = self.bn1(outputs)
        outputs = self.relu(outputs)
        
        # 经过第二个卷积层
        outputs = self.conv2(outputs)
        outputs = self.bn2(outputs)
        
        # 该残差块的最后输出
        identity = self.shortcut(identity)
        outputs = outputs + identity
        final_out = self.relu(outputs)
        
        return final_out

In [16]:
class ResNet18(nn.Module):
    def __init__(self, num_classes=1000):
        #3 继承类
        super().__init__()
        
        ## 起始卷积层和池化层
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # 定义残差块1
        self.reslayer1 = self._make_layer(64, 64, 2, 1)
        # 定义残差块2
        self.reslayer2 = self._make_layer(64, 128, 2, 2)
        # 定义残差块3
        self.reslayer3 = self._make_layer(128, 256, 2, 2)
        # 定义残差块4
        self.reslayer4 = self._make_layer(256, 512, 2, 2)
        
        ## 最后的均值池化层和全连接层
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, num_classes)
        
    
    def _make_layer(self, in_channels, out_channels, blocks_num, stride=1):
        layer = []
        layer.append(ResidualBlock(in_channels, out_channels, stride))
        
        for i in range(1, blocks_num):
            layer.append(ResidualBlock(out_channels, out_channels, stride))
            
        return nn.Sequential(*layer)
        
        
    def forward(self, inputs):
        # 经过起始的第一成卷积层
        outputs = self.conv1(inputs)
        outputs = self.bn1(outputs)
        outputs = self.relu(outputs)
        outputs = self.maxpool(outputs)
        
        # 进入1,2,3,4的残差块
        outputs = self.reslayer1(outputs)
        outputs = self.reslayer2(outputs)
        outputs = self.reslayer3(outputs)
        outputs = self.reslayer4(outputs)
        
        # 进入均值池化和全连接层
        outputs = self.avgpool(outputs)
        outputs = self.fc(torch.flatten(outputs, 1))
        
        return outputs
        

In [17]:
model = ResNet18()
print(model)

ResNet18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (reslayer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (c

In [18]:
from torchsummary import summary
summary(model, (3,244,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,472
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]          36,928
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,928
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
    ResidualBlock-11           [-1, 64, 61, 61]               0
           Conv2d-12           [-1, 64, 61, 61]          36,928
      BatchNorm2d-13           [-1, 64, 61, 61]             128
             ReLU-14           [-1, 64,